In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
# R = 2
# L_smooth = 0.1 #np.inf
# Inductance = 4
# Capacitance = 4 # 5

mu = 0.

# R, L_smooth, Inductance, Capacitance = 3, 0.1, 3, 8/3
# R, L_smooth, Inductance, Capacitance = 2, 0.1, 4, 4
# R, L_smooth, Inductance, Capacitance = 1, 0.1, 8, 8
# R, L_smooth, Inductance, Capacitance = 0.5, 0.1, 16, 16
# R, L_smooth, Inductance, Capacitance = 1, 0.1, 6, 8

# R, L_smooth, Inductance, Capacitance = 0.5, 0.01, 11, 24

R, L_smooth, Inductance, Capacitance = 1.5, 0.1, 4.5, 6


solver = "ipopt"
# solver = "ipopt_qcqp"
# solver = "ipopt_qcqp_matrix"

In [3]:
### Case that worked
# R, L_smooth, Inductance, Capacitance = 2, 0.1, 4, 4  # took 10 min
# R, L_smooth, Inductance, Capacitance = 1, 0.1, 8, 8  # took 1 hour
# R, L_smooth, Inductance, Capacitance = 1, 0.1, 6, 8 # took 28m, problem.obj = problem.b
# R, L_smooth, Inductance, Capacitance = 2, 0.01, 3, 4 # took 12m, problem.obj = problem.b, fails verification

# Ciropt problem

In [278]:
print(f"{mu=}, {L_smooth=}, {R=}, {Capacitance=}, {Inductance=}, {solver=}")

mu=0.0, L_smooth=0.1, R=1.5, Capacitance=6, Inductance=4.5, solver='ipopt'


In [287]:
problem = co.dadmm_C_graph6( mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d 
# problem.obj = problem.b

bounds = {  # 'd': {"ub": 100, "lb": 1e-4},
            # 'b': {"ub": 100, "lb": 1e-4},
            # 'h': {"ub": 100, "lb": 1e-4},
            # 'gamma': {"ub": 10, "lb": 1e-5}
            }

# res, sol = problem.solve(solver=solver, bounds=bounds, extra_dim=530, verbose=True)[:2]
res, sol = problem.solve(solver=solver, extra_dim=530, verbose=False)[:2]

ca_vars = problem.vars
print(res["b"])

dim_G=29, dim_F=12


In [286]:
res

{'b': 3.8361256468330485,
 'h': 3.8361257015293804,
 'd': -7.352231050516314e-09,
 'gamma': 2.5574171343500285}

In [164]:
# mu=0.0, L_smooth=0.1, R=2, Capacitance=4, Inductance=4, solver='ipopt'
# {'b': 2.7644089027398016, 'h': 2.764408907848405, 'd': 0.07044237664742883, 'gamma': 1.3822044539230276} 

# mu=0.0, L_smooth=0.5, R=50, Capacitance=1, Inductance=10, solver='ipopt_qcqp'
# {'b': 0.3978152863640579, 'h': 0.3978152934397974, 'd': 0.0003904411732310108, 'alpha': 0.0, 'beta': 0.0, 'gamma': 0.007956305939653716, 'delta': 0.0}

# mu=0.0, L_smooth=0.5, R=18, Capacitance=1, Inductance=5, solver='ipopt_qcqp'
# {'b': 0.5425479142544587, 'h': 0.5425477172429249, 'd': 0.0018205564359045578, 'alpha': 0.0, 'beta': 0.0, 'gamma': 0.030141488274104795, 'delta': 0.0}

# mu=0.0, L_smooth=0.5, R=18, Capacitance=1, Inductance=10, solver='ipopt_qcqp'
# {'b': 1.0541047016940057, 'h': 1.0541046971920436, 'd': 0.005040260539486683, 'alpha': 0.0, 'beta': 0.0, 'gamma': 0.05856137206621561, 'delta': 0.0}

# PEP verification

In [283]:
# res["b"] = 1.
res["d"] = 0
# res["d"] = -9.040457075280738e-09

In [284]:
problem = co.dadmm_C_graph6( mu, L_smooth, R, Capacitance, Inductance, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=7.0846954969283615e-09


# Just for test

In [58]:
# R, L_smooth, Inductance, Capacitance = 1, 0.1, 8, 8
# res = { 'b': 5.5409883698753974, 'h': 5.5409883749567, 'd': 0.17881074671227212, 'gamma': 5.540988374956701 }

# R, L_smooth, Inductance, Capacitance = 2, 0.1, 4, 4
# res = {'b': 2.7644089027398016, 'h': 2.764408907848405, 'd': 0.07044237664742883, 'gamma': 1.3822044539230276 }

# R, L_smooth, Inductance, Capacitance = 2, 0.1, 3, 4
# res = { 'b': 2.3, 'h': 2.3, 'd': 0, 'gamma': 1.15 }

In [10]:
R, L_smooth, Inductance, Capacitance = 2, 100, 4, 4
res = { 'b': 2.7644089078484, 'h': 2.7644089078484, 'd': 0, 'gamma': 1.38220445392302 }

In [11]:
# R, L_smooth, Inductance, Capacitance = 2, 0.1, 4, 4
# res = {'b': 1, 'h': 2.764408907848405, 'd': 0, 'gamma': 2.764408907848405 }

# R, L_smooth, Inductance, Capacitance = 3, 1000, 4, 8/9
# res = { 'b': 4/3, 'h': 4/3, 'd': 0, 'gamma': 4/9 }

# R, L_smooth, Inductance, Capacitance = 3, 10**(100), 4, 8/9
# res = { 'b': 4/3, 'h': 4/3, 'd': 0, 'gamma': 4/9 }

# R, L_smooth, Inductance, Capacitance = 2, 100, 4, 4
# res = { 'b': 2.7644089078484, 'h': 2.7644089078484, 'd': 0, 'gamma': 1.38220445392302 }

In [12]:
problem = co.dadmm_C_graph6( mu, L_smooth, R, Capacitance, Inductance, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

UserWarning: PEPit didn't find any nontrivial worst-case guarantee. It seems that the optimal value of your problem is unbounded.